In [38]:
import numpy as np
import matplotlib as plt
from scipy import optimize

# Tarea 4: Restauración de imagenes

## Instrucciones

* La tarea es individual.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta utilizacion de algoritmos de forma vectorizada.
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el viernes 31 de Julio a las **18:00 hrs**.  Se aceptarán entregas hasta las 19:00 hrs sin descuento en caso de existir algun problema, posteriormente existirá un descuento lineal hasta las 20:00 hrs del mismo día.
* Las tareas que sean entregadas antes del jueves a mediodía recibirán una bonificación de 10 puntos
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).

# Introducción

Como se vio en la tarea anterior la interpolación bicúbica sirve para aumentar la dimensión de una imagen obteniendo valores de nuevos pixeles interiores. En esta tarea se verá otra aplicación a la interpolación bicúbica para la restauración de imágenes

# Eliminación de ruido 


Supongamos que se tiene una imagen $X$ cuyos pixeles presentan valores $I_{xy}$ con ruido y se quiere volver a la imagen original


In [39]:

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error
from skimage import io
from skimage import color

img = color.rgb2gray(io.imread('cat_4pixel.png'))

img = img_as_float(img)

rows, cols = img.shape


noise = np.ones_like(img) * 0.2 * (img.max() - img.min())
noise[np.random.random(size=noise.shape) > 0.5] *= -1

img_noise = img + noise

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4),
                         sharex=True, sharey=True)
ax = axes.ravel()


ax[0].imshow(img, cmap=plt.cm.gray, vmin=0, vmax=1)

ax[0].set_title('Original image')

ax[1].imshow(img_noise, cmap=plt.cm.gray, vmin=0, vmax=1)
ax[1].set_title('Image with noise')

plt.tight_layout()
plt.show()

/home/jorge/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  import sys


AttributeError: module 'matplotlib' has no attribute 'subplots'

El método a implementar, será utilizando la interpolación bicúbica que viene representada por la siguiente spline

$$
p(x,y) = \sum\limits_{i=0}^3 \sum_{j=0}^3 a_{ij} x^i y^j.
$$
Como se puede observar el problema de interpolación se resume en determinar los 16 coeficientes $a_{ij}$. Ya que la imagen presenta ruido no se puede despejar los coeficientes asumiendo que los valores de las derivadas $f_x$, $f_y$ y $f_{xy}$ son correctas. 

Para determinar los coeficientes se seleccionara secciones solapadas de la imagen de $5\times5 $ pixeles y se determinará una función $p_i$ por cada sección, ejemplificando con una imagen de $9\times 9$  las secciones quedarían de la siguiente manera:


<img src="matriz_seccion.png" width="50%"/>

Se busca la minimización del error cuadrático entre la spline generada $p_i$ y el punto de la imagen, en el caso del ejemplo anterior la función objetivo a minimizar es:

$$(p_1(0,0)-I_{00})^2 + (p_1(0,1)-I_{01})^2 +\dots+(p_1(4,4)-I_{44})^2 + (p_2(4,4)-I_{44})^2+ \dots+(p_4(8,8)-I_{88})^2 $$


Ya que la imagen debe presentar continuidad entre las secciones los pixeles que abarcan más de una deben presentar el mismo valor por lo tanto existen restricciones que se deben cumplir:

$$p_1(0,4) = p_2(0,4)$$
$$p_1(4,4) = p_2(4,4)$$
$$p_1(4,0) = p_3(4,0)$$
$$p_1(4,4) = p_3(4,4)$$
$$\vdots$$

Es decir los valores de los pixeles de solamente las **esquinas** deben ser iguales en todas las secciones que coinciden 

Finalmente al obtener los valores de los coeficientes la nueva imagen será el resultado de la evaluación de todos los pixeles en su spline respectiva

# Preguntas

## 1.Restauración de imagenes

### 1.1 Generar $p_i(x,y)$ (10 pts)

Debe implementar la función `spline_evaluate` que reciba un arreglo con los valores de los coeficientes y el valor de la coordenada $x$ e $y$ del pixel y debe retornar el valor del pixel evaluado en la spline


In [15]:
def spline_evaluate(a,pos):
    '''
    a: (array 16) arreglo de coeficientes
    pos: (tuple) tupla con forma (x,y) que representa la posicion del pixel
    
    return
    value: (float) evaluacion del pixel
      
    '''
    value = 0
    a = a.reshape((4,4))
    for i in range(4):
        for j in range(4):
            value += a[i,j] * (pos[0]**i) * (pos[1]**j) 
    return value
    

### 1.2 Generar funcion a minimizar (25 pts)

Debe implementar la función `objective_function` que reciba un arreglo con los valores de todos los coeficientes necesarios y la imagen con ruido, y debe retornar el error cuadrático entre el polinomio y el valor del pixel de la imagen.


In [16]:
def generate_matrix(n):
    A = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            A[i][j] = (i * j) % 50
    return A

def generate_coef(n):
    a = np.zeros(16 * n)
    for i in range(16 * n):
        a[i] = (i * i) % 20
    return a

In [68]:
def objective_function(a,image):
    '''
    a: (array) array con todos los coeficientes 
    image: (array nxn) imagen que presenta ruido en sus datos
    
    return
    error: suma total del error cuadratico entre la spline evaluada y el valor del pixel respectivo
    
    '''
    n = image.shape[0]
    l = []
    spli = a.shape[0]/16
    j = 0
    q = 0
    for i in range(int(spli)):
            for z in range(0, 5):
                for v in range(0, 5):
                    val = (spline_evaluate(a[i*16:i*16+16],(z+j,v+q)) - image[z+j,v+q])**2
                    l.append(val)
            j+=4
            if(j >= n-1):
                j = 0
                q += 4
    error = sum(l)/len(l)
    return error
    
    

In [69]:
image = generate_matrix(13)
a = generate_coef(9)

In [70]:
objective_function(a,image)

20081381337023.324

### 1.3 Generar Restricciones (25 pts)

Se debe implementar la función `create_constraints` que reciba la imagen y retorne una lista de diccionarios con las restricciones del problema. El diccionario debe tener la siguiente estructura:

`{"type": "eq", "fun": funcion_con_restriccion}`



In [71]:
def create_constraints(image):
    '''
    image: (array nxn) imagen que presenta ruido en sus datos
    
    return
    constraints: (list) lista de restricciones  
    '''
    constraints = []
    n = image.shape[0]
    spline = (n-5)/4 + 1
    inter = int(spline-1)
    af = np.linspace(4, 4*inter, 4)
    for i in af:
        for j in af:
            f1 = lambda a1,a2 : spline_evaluate(a1,(i,j)) == spline_evaluate(a2,(i,j))
            dic = {"type": "eq", "fun": f1}
            constraints.append(dic)
            f2 = lambda a1,a2 : spline_evaluate(a1,(j,i)) == spline_evaluate(a2,(j,i))
            dic = {"type": "eq", "fun": f2}
            constraints.append(dic)
    return constraints

In [72]:
create_constraints(image)

[{'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>(a1, a2)>},
 {'type': 'eq',
  'fun': <function __main__.create_constraints.<locals>.<lambda>

### 1.3 Generar nueva imagen (30 pts)
Se debe implementar la función `clean_image` que reciba un arreglo con una imagen en escala de grises y si es que se consideran las restricciones de continuidad. Debe retornar una nueva imagen con el ruido minimizado. 
Para realizar la minimización, debe utilizarse la función [minimize de la libreria de scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html)


In [90]:
def clean_image(image,constraints):
    '''
    image: (array nxn) imagen con ruido
    constraints: (bool) true si es que se consideran las restricciones
    
    return
    new_iamge: (array nxn) imagen restaurada
    '''
    n = image.shape[0]

    new_image = optimize.minimize(objective_function,image.reshape(n*n))
    
    return new_image

In [91]:
clean_image(image,False)

TypeError: objective_function() missing 1 required positional argument: 'image'

## 2. Evaluar error (5 pts)

Implemente la función `error_restore` la cual debe obtener el error de la imagen obtenida comparándola con una de referencia. El error debe ser calculado utilizando el índice SSIM (Structural similarity)

In [56]:
def error_restore(original,new):
    """
    Parameters
    ----------
    image:	(nxn array) imagen original limpia
    new:	(nxn array) imagen restaurada


    Returns
    -------
    error:	(float) diferencia entre imagenes (1-ssim) 

    """
    n = original.shape[0]
    error = 1-structural_similarity(original, new, multichannel=True, data_range=n)
    return error

**Pregunta: ¿Como afecta el uso de restricciones en el error?** (5 pts)

# Consideraciones

* Se trabajará con imágenes cuadradas en escala de grises
* Las imágenes tendrán una dimensión adecuada para que no sobren o falten pixeles para la agrupación de $5\times5$
* Para los casos de los pixeles que no tienen restricción pero pueden ser evaluados por distintas splines considere el valor de la spline de la izquierda o superior
